## Compare PGS scores
Compare output from the different PGS prediction models.

In [ ]:
import matplotlib
matplotlib.use('nbagg')

In [ ]:
%matplotlib inline

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
matplotlib.rc_file_defaults()

In [ ]:
def ztransform(x):
    '''
    Return input normalized to zero mean and unit variance
    
    Parameters
    ----------
    x: ndarray
    
    Returns
    -------
    ndarray
    '''
    return (x - x.mean()) / x.std()

In [ ]:
# make some guesses where reference data is located in relation to host
if os.path.isdir('/REF'):  # mounted dir on container
    REF = '/REF'
elif 'REFERENCE' in os.environ:
    REF = os.environ['REFERENCE']
else:
    REF = os.path.join('..', 'reference')

In [ ]:
# Load phenotypes
Pheno_file = os.path.join(REF, 'examples/prsice2/EUR.height')
pheno = pd.read_csv(Pheno_file, delim_whitespace=True)
pheno_type = pheno.columns[2]

In [ ]:
# plink
Output_dir = os.path.join('results', 'PGS_plink')
Data_prefix = 'EUR'

# scores
scores_plink = pd.read_csv(os.path.join(Output_dir, 'test.score'), sep=' ')
metrics_plink = pd.read_csv(os.path.join(Output_dir, 'test_summary.csv'), sep=' ')

# merge phenotype and score dataframes
pheno_scores_plink = pd.merge(pheno, scores_plink, on=["FID", "IID"])

# plot SCORE vs measured HEIGHT
plt.plot(pheno_scores_plink['score'], pheno_scores_plink[pheno_type], '.')
plt.xlabel('$PGS_\mathrm{Plink}$')
plt.ylabel(pheno_type)

In [ ]:
# PRSice-2
Output_dir = os.path.join('results', 'PGS_prsice2')
Data_prefix = 'EUR'

# scores
scores_prsice2 = pd.read_csv(os.path.join(Output_dir, 'test.score'), sep=' ')
metrics_prsice2 = pd.read_csv(os.path.join(Output_dir, 'test_summary.csv'), sep=' ')

# merge phenotype and score dataframes
pheno_scores_prsice2 = pd.merge(pheno, scores_prsice2, on=["FID", "IID"])

# plot SCORE vs HEIGHT
plt.plot(pheno_scores_prsice2['score'], pheno_scores_prsice2[pheno_type], '.')
plt.xlabel('$PGS_\mathrm{PRSice2}$')
plt.ylabel(pheno_type)

In [ ]:
# LDpred2 infinitesimal model
Output_dir = os.path.join('results', 'PGS_LDpred2_inf')

# scores
pheno_scores_ldpred2_inf = pd.read_csv(os.path.join(Output_dir, 'test.score'), delim_whitespace=True)
metrics_ldpred2_inf = pd.read_csv(os.path.join(Output_dir, 'test_summary.csv'), sep=' ')

# merge phenotype and score dataframes
# pheno_scores_ldpred2_inf = pd.merge(pheno, scores_ldpred2_inf, on=["FID", "IID"], suffixes=[None, '_y'])

# plot SCORE vs HEIGHT
plt.plot(pheno_scores_ldpred2_inf['score'], pheno_scores_ldpred2_inf[pheno_type], '.')
plt.xlabel('$PGS_\mathrm{ldpred2\_inf}$')
plt.ylabel(pheno_type)

In [ ]:
# LDpred2 automatic model
Output_dir = os.path.join('results', 'PGS_LDpred2_auto')

# scores
pheno_scores_ldpred2_auto = pd.read_csv(os.path.join(Output_dir, 'test.score'), delim_whitespace=True)
metrics_ldpred2_auto = pd.read_csv(os.path.join(Output_dir, 'test_summary.csv'), sep=' ')

# merge phenotype and score dataframes
# pheno_scores_ldpred2_auto = pd.merge(pheno, scores_ldpred2_auto, on=["FID", "IID"], suffixes=[None])

# plot SCORE vs HEIGHT
plt.plot(pheno_scores_ldpred2_auto['score'], pheno_scores_ldpred2_auto[pheno_type], '.')
plt.xlabel('$PGS_\mathrm{ldpred2\_auto}$')
plt.ylabel(pheno_type)

In [ ]:
# compare scores and distributions
all_scores = [
    pheno_scores_plink, 
    pheno_scores_prsice2, 
    pheno_scores_ldpred2_inf, 
    pheno_scores_ldpred2_auto
    ]
all_metrics = [
    metrics_plink,
    metrics_prsice2,
    metrics_ldpred2_inf,
    metrics_ldpred2_auto
]
labels = [
    r'$PGS_\mathrm{Plink}$',
    r'$PGS_\mathrm{PRSice2}$', 
    r'$PGS_\mathrm{LDpred2-inf}$', 
    r'$PGS_\mathrm{LDpred2-auto}$'
    ]

fig, axes = plt.subplots(len(all_scores) + 1, len(all_scores), figsize=(12, 10), sharex='col')
fig.subplots_adjust(wspace=0.4, hspace=0.4)

for i in range(len(all_scores)):
    y = all_scores[i]['score']
    for j in range(len(all_scores)):
        x = all_scores[j]['score']
        if i < j:
            axes[i, j].set_visible(False)
        elif i==j:
            axes[i, j].hist(x, bins=51)
            axes[i, j].set_ylabel('count', labelpad=0)
        elif i > j:
            sns.kdeplot(x=x, 
                        y=y,
                        ax=axes[i, j],
                        color='gray')
            axes[i, j].plot(x, y, 'C0.', ms=2)
            axes[i, j].set_ylabel(labels[i], labelpad=0)
            axes[i, j].set_title(f'CC={np.corrcoef(x, y)[1, 0]:.3f}')

# phenotype vs. PGS
for j in range(len(all_scores)):
    ax = axes[-1, j]
    x = all_scores[j]['score']
    y = all_scores[j][pheno_type]
    sns.kdeplot(x=x, 
                y=y,
                ax=ax,
                color='gray')
    ax.plot(x, y, '.', ms=2)
    ax.set_xlabel(labels[j], labelpad=0)
    ax.set_ylabel(pheno_type)
    title = '\n'.join([
        # f'CC={np.corrcoef(x, y)[1, 0]:.3f}',
        f'$\mathrm{{LM}}_{{R^2}}$={all_metrics[j]["r.squared"].to_numpy()[0]:.3f}'
    ])
    ax.set_title(title)

In [ ]:
# normalized scores
fig, axes = plt.subplots(len(all_scores) + 1, len(all_scores), figsize=(12, 10), sharex='col')
fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.suptitle(f'normalized scores ($\mu=0, \sigma^2=1$)')

for i in range(len(all_scores)):
    y = ztransform(all_scores[i]['score'])
    for j in range(len(all_scores)):
        x = ztransform(all_scores[j]['score'])
        if i < j:
            axes[i, j].set_visible(False)
        elif i==j:
            axes[i, j].hist(x, bins=51)
            axes[i, j].set_ylabel('count', labelpad=0)
        elif i > j:
            sns.kdeplot(x=x, 
                        y=y,
                        ax=axes[i, j],
                        color='gray')
            axes[i, j].plot(x, y, 'C0.', ms=2)
            axes[i, j].set_ylabel(labels[i], labelpad=0)
            axes[i, j].set_title(f'CC={np.corrcoef(x, y)[1, 0]:.3f}')

# height vs. PGS
for j in range(len(all_scores)):
    ax = axes[-1, j]
    x = ztransform(all_scores[j]['score'])
    y = ztransform(all_scores[j][pheno_type])
    sns.kdeplot(x=x, 
                y=y,
                ax=ax,
                color='gray')
    ax.plot(x, y, '.', ms=2)
    ax.set_xlabel(labels[j], labelpad=0)
    ax.set_ylabel(pheno_type)
    title = '\n'.join([
        #f'CC={np.corrcoef(x, y)[1, 0]:.3f}',
        f'$\mathrm{{LM}}_{{R^2}}$={all_metrics[j]["r.squared"].to_numpy()[0]:.3f}'
    ])
    ax.set_title(title)